# SQLとSQLite入門: データベースを操作しよう

このノートブックでは、プログラミングでデータを保存・管理するための言語 **SQL (Structured Query Language)** の基本と、手軽に使えるデータベース **SQLite** の使い方を学ぶ。

### 学ぶこと

1. **データベースの作成**: データの入れ物を作る
2. **テーブルの作成**: データを入れる表を作る
3. **データの追加 (INSERT)**: データを書き込む
4. **データの取得 (SELECT)**: データを読み込む **(最重要！)**
5. **データの更新・削除**: データを書き換える・消す

In [ ]:
import sqlite3
import pandas as pd # データをきれいに表示するために使用する

# 1. データベースに接続する
# 'sample_db.sqlite' というファイルがなければ作成され、あれば接続される。
dbname = 'sample_db.sqlite'
conn = sqlite3.connect(dbname)

# カーソル（データベースを操作する「手」のようなもの）を作成
cur = conn.cursor()

print(f"{dbname} に接続しました。")

## 1. テーブル（表）の作成

データベースの中に、データを管理するための「表（テーブル）」を作ります。
ここでは、社員名簿のような `employees` テーブルを作ってみましょう。

* `id`: 社員番号（整数、自動で割り振られる主キー）
* `name`: 名前（文字列）
* `age`: 年齢（整数）
* `department`: 部署（文字列）

SQLでは `CREATE TABLE` という命令を使います。

In [ ]:
# SQL文の作成
# IF NOT EXISTS は「もしテーブルがまだ無ければ作る」という意味です。
create_table_query = """
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    department TEXT
);
"""

# SQLを実行
cur.execute(create_table_query)

print("テーブル 'employees' が作成されました（すでに存在する場合はスキップ）。")

## 2. データの追加 (INSERT)

作成したテーブルにデータを入れてみます。
`INSERT INTO テーブル名 (カラム名...) VALUES (値...)` という構文を使います。

**重要**: 最後に `conn.commit()` をしないと、変更が確定（保存）されません。

In [ ]:
# 1件ずつ追加する場合
cur.execute("INSERT INTO employees (name, age, department) VALUES ('佐藤 太郎', 30, '営業部')")

# 複数のデータを一度に追加する場合
data_list = [
    ('鈴木 花子', 24, '人事部'),
    ('田中 次郎', 45, '開発部'),
    ('高橋 優子', 29, '営業部'),
    ('伊藤 健太', 35, '開発部')
]

# executemany を使うとまとめて追加できます
cur.execute("INSERT INTO employees (name, age, department) VALUES (?, ?, ?)", data_list[0]) # 1つ目の例
cur.executemany("INSERT INTO employees (name, age, department) VALUES (?, ?, ?)", data_list[1:]) # 残りの例

# 変更を保存（コミット）
conn.commit()

print("データを追加して保存しました。")

## 3. データの取得 (SELECT)

データベースに入っているデータを取り出します。これがSQLで最もよく使う操作です。
`SELECT * FROM テーブル名` で「そのテーブルの全ての列」を取得します。

In [ ]:
# SQLを実行してデータを取得
cur.execute("SELECT * FROM employees")

# 取得したデータをすべて取り出す
results = cur.fetchall()

# Pythonのリストとして表示
print("--- Pythonのリスト形式で表示 ---")
for row in results:
    print(row)

print("\n--- Pandasを使ってきれいに表示 ---")
# Pandasを使うと表形式で見やすくなります
df = pd.read_sql("SELECT * FROM employees", conn)
display(df)

### 条件を指定して検索 (WHERE句)

「営業部の人だけ」「30歳以上」など、条件を絞り込むには `WHERE` を使う。

In [ ]:
# 開発部 (department = '開発部') の社員だけを取得
sql_query = "SELECT * FROM employees WHERE department = '開発部'"

df_dev = pd.read_sql(sql_query, conn)
display(df_dev)

## 4. データの更新と削除 (UPDATE / DELETE)

* **UPDATE**: データを書き換えます。
* **DELETE**: データを削除します。

※ `WHERE` を忘れると、全てのデータが更新されたり削除されたりするので注意が必要です！

In [ ]:
# 【更新】佐藤さんの年齢を 31歳 に変更する
cur.execute("UPDATE employees SET age = 31 WHERE name = '佐藤 太郎'")

# 【削除】鈴木さんを名簿から削除する
cur.execute("DELETE FROM employees WHERE name = '鈴木 花子'")

# 変更を保存
conn.commit()

# 結果を確認
print("更新・削除後のテーブル:")
display(pd.read_sql("SELECT * FROM employees", conn))

## 終了処理

データベースを使い終わったら、必ず接続を閉じます。

In [ ]:
# 接続を閉じる
cur.close()
conn.close()

print("データベース接続を閉じました。お疲れ様でした！")

# 練習問題： 
以下の手順でコードを書け。

データベース接続: library.sqlite というファイル名で接続する。

テーブル作成: books テーブルを作る（ID, タイトル, 著者, 価格）。

データ登録: 3〜4冊の本を登録する。

データ検索: 2000円以上の本だけを表示する。

データ更新: 特定の本の価格を書き換える。

データ削除: 不要なデータを1件消す。